# k-means法
クラスタリングと呼ばれる手法にあたり、データを複数のクラスタに分けて大まかな特徴を捉える際に使用する。

1. 人間側がクラスタの数を決める
2. ランダムに振られた点（重心）から近いものをクラスタとする
3. 紐づいたクラスタとの距離を元に点を移動させる

In [1]:
import pandas as pd
df = pd.read_csv('file/convinience_store.csv')
df.head()

,No,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ
0,1,25350,3650,8945,0,4867,8945
1,2,24500,0,0,1827,0,0
2,3,23050,5750,11570,0,7667,11570
3,4,22850,4100,10145,0,5467,10145
4,5,22500,0,0,0,0,0


In [2]:
# No列を削除してNumpy配列に変換する
x = df.drop('No', axis=1).values
x[:3]

array([[25350,  3650,  8945,     0,  4867,  8945],
       [24500,     0,     0,  1827,     0,     0],
       [23050,  5750, 11570,     0,  7667, 11570]])

In [3]:
from sklearn.cluster import KMeans

In [4]:
# クラスタ数は、ドメイン知識や業務上の必要性から決定する
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(x)

KMeans(n_clusters=3, random_state=0)

k-meansのモデルの学習では、それぞれのクラスターの中心座標を算出して保持する。

今回のデータセットは6次元で、クラスター数を3と指定したため、3行6列の値を確認することができる。

In [5]:
# 各クラスターの中心座標
kmeans.cluster_centers_

array([[ 5043.5483871 ,  3486.41935484,  7017.74193548,  2206.        ,
         2512.90322581,  3509.12903226],
       [28681.25      ,  5637.66666667,  1298.75      ,  1271.        ,
         1770.875     ,   716.95833333],
       [17266.66666667,  2730.93333333, 10721.66666667,   590.86666667,
         3447.33333333,  9851.2       ]])

In [6]:
# 予測値の取得
cluster = kmeans.predict(x)
cluster

array([2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 0,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1], dtype=int32)

In [7]:
# 元のデータセットにクラスタ情報の列を追加
df_cluster = df.copy()
df_cluster['cluster'] = cluster

In [8]:
df_cluster.head()

,No,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ,cluster
0,1,25350,3650,8945,0,4867,8945,2
1,2,24500,0,0,1827,0,0,1
2,3,23050,5750,11570,0,7667,11570,2
3,4,22850,4100,10145,0,5467,10145,2
4,5,22500,0,0,0,0,0,1


In [9]:
# 結果の検証
# まず空のDataFrameを作る
df_results = pd.DataFrame()
df_results

""


In [10]:
# クラスタが0の行のみを取得
df_cluster[df_cluster['cluster'] == 0].head()

,No,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ,cluster
18,19,14500,8775,0,1003,11700,0,0
21,22,10750,0,0,3413,0,0,0
24,25,10600,4363,6695,1860,2908,3348,0
25,26,10200,5125,7970,1350,3417,3985,0
26,27,10000,7375,0,2360,4917,0,0


In [11]:
# クラスタ0の平均値
df_cluster[df_cluster['cluster'] == 0].mean()

No               37.806452
弁当・麺類          5043.548387
飲料             3486.419355
おにぎり・サンドイッチ    7017.741935
スイーツ           2206.000000
カップスープ         2512.903226
サラダ            3509.129032
cluster           0.000000
dtype: float64

In [12]:
# df_resultsにクラスタごとの平均値を入れる
df_results['cluster 0'] = df_cluster[df_cluster['cluster'] == 0].mean().tolist()
df_results['cluster 1'] = df_cluster[df_cluster['cluster'] == 1].mean().tolist()
df_results['cluster 2'] = df_cluster[df_cluster['cluster'] == 2].mean().tolist()

In [13]:
df_results

,cluster 0,cluster 1,cluster 2
0,37.806452,46.250000,13.533333
1,5043.548387,28681.250000,17266.666667
2,3486.419355,5637.666667,2730.933333
3,7017.741935,1298.750000,10721.666667
4,2206.000000,1271.000000,590.866667
5,2512.903226,1770.875000,3447.333333
6,3509.129032,716.958333,9851.200000
7,0.000000,1.000000,2.000000


In [14]:
# indexを数字でなくラベルを割り当てる
df_results = df_results.set_index(df_cluster.columns)
df_results

,cluster 0,cluster 1,cluster 2
No,37.806452,46.250000,13.533333
弁当・麺類,5043.548387,28681.250000,17266.666667
飲料,3486.419355,5637.666667,2730.933333
おにぎり・サンドイッチ,7017.741935,1298.750000,10721.666667
スイーツ,2206.000000,1271.000000,590.866667
カップスープ,2512.903226,1770.875000,3447.333333
サラダ,3509.129032,716.958333,9851.200000
cluster,0.000000,1.000000,2.000000


In [15]:
# 不要なカラムを削除
# .Tで行と列を反転
df_results = df_results.drop(['No', 'cluster']).T
df_results

,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ
cluster 0,5043.548387,3486.419355,7017.741935,2206.000000,2512.903226,3509.129032
cluster 1,28681.250000,5637.666667,1298.750000,1271.000000,1770.875000,716.958333
cluster 2,17266.666667,2730.933333,10721.666667,590.866667,3447.333333,9851.200000
